In [15]:
import numpy as np

In [16]:
def initialize():
    global T, J, JR, γ, egam, β, α, δ, ξ, υ, ϵ, μ, tol, damp, itermax,\
    w, r, wn, winc, Rn, p, K, ma, n_e,\
    L, A, C, I, B, G, k, Y, Bf, Tb, TXr, Tpen, H, eps, n_p, gy, by, κ,\
    smopec, τk, τc, τw, τr, τp, τs, tax, pen, a, e, c, m


    T = 24; J = 3; JR = 3; γ = .5; egam = 1.0 - 1.0/γ; β = .9
    α = .3; δ = .0; ξ = 2.0; υ = 0.5; ϵ = 0.0
    μ = 0.8; tol = 1e-6; damp = 0.3; itermax = 200


    w  = np.zeros(T+1); r = np.zeros(T+1); wn = np.zeros(T+1)
    winc = np.zeros((J,T+1)); ma = np.zeros((J,T+1))
    Rn = np.zeros(T+1); p = np.zeros(T+1); K  = np.zeros(T+1)
    L  = np.zeros(T+1); A = np.zeros(T+1); C  = np.zeros(T+1)
    I  = np.zeros(T+1); B = np.zeros(T+1); G  = np.zeros(T+1)
    k  = np.zeros(T+1); Y = np.zeros(T+1); Bf = np.zeros(T+1)
    Tb = np.zeros(T+1); TXr = np.zeros(T+1); Tpen = np.zeros(T+1)
    H = np.ones(T+1); eps = np.zeros(T+1)


    n_p = np.zeros(T+1)
    n_e = np.zeros(T+1)
    gy  = 0.0
    by  = np.zeros(T+1)
    κ   = np.zeros(T+1)
    smopec = False


    τk  = np.zeros(T+1)
    τc = np.zeros(T+1)
    τw = np.zeros(T+1)
    τr = np.zeros(T+1)
    τp = np.zeros(T+1)
    τs = np.zeros(T+1)
    tax = np.ones(T+1)
    pen = np.zeros((J,T+1))


    a = np.zeros((J,T+1))
    e = np.zeros(T+1)
    c = np.zeros((J,T+1))


    m = np.zeros((J,T+1))
    for t in range(T+1):
        m[0,t] = 1.0
        tm = year(t, 1, 0)
        for j in range(1,J):
            m[j,t] = m[j-1,tm]/(1.0 + n_p[t])
    return None

In [17]:
def year(t, j, jp):
    year = t + jp - j
    if t == 0 or year<=0:
        year = 0
    if t == T or year>=T:
        year = T
    return year

In [18]:
def factor_prices(t):
    global k, r, w, wn, winc, Rn, p
    
    k[t]   = K[t]/L[t]                
    
    if smopec and t > 0:
        r[t] = r[0]
    else:
        r[t] = (1.0 - τk[t])/(1.0 - eps[t]*τk[t])*(α*k[t]**(α-1.0)*H[t]**ϵ-δ)
  
    w[t]   = (1.0 - α)*k[t]**α*H[t]**ϵ

    wn[t]  = w[t]*(1.0 - τw[t] - τp[t])
    winc[0,t] = (1.0-(1.0-τs[t])*e[t])*wn[t]
    
    for j in range(1, JR-1):
        tp = year(t, j , 1)
        winc[j,t] = wn[t]*(1.0+n_e[tp])/μ
    
    Rn[t]  = 1.0 + r[t]*(1.0 - τr[t])  
    p[t]   = 1.0 + τc[t]         
    return None

In [19]:
def get_W(j, t):
    
    Assets = winc[j,t] + pen[j,t]
    
    if t == 1 and j > 0:
        Assets = Assets + Rn[t] * a[j,t]
        
    PRn = 1.0
    for jp in range(j+1,J):
        tp = year(t,j,jp)
        PRn = PRn*Rn[tp]
        Assets = Assets + (winc[jp,tp] + pen[jp,tp])/PRn
    return Assets


def get_Psi(j,t):
    Psi = 1.0
    PRn = 1.0
    
    for jp in range(j+1,J):
        tp = year(t, j ,jp)
        PRn = PRn*Rn[tp]
        Psi = Psi + β**((jp-j)*γ)*(p[tp]/(PRn*p[t]))**(1.0-γ)
    Psi = 1.0/(p[t]*Psi)
    
    return Psi

In [20]:
def get_path(j,t):
    
    global c, a
    
    PRn = 1.0

    for jp in range(j+1,J):
        
        tp = year(t,j,jp)
        tm = year(t,j,jp-1)
        PRn = PRn*Rn[tp]
        

        c[jp,tp] = (β**(jp-j)*PRn*p[t]/p[tp])**γ*c[j,t]
        a[jp,tp] = winc[jp-1,tm] + pen[jp-1,tm] + Rn[tm]*a[jp-1,tm] - p[tm]*c[jp-1,tm]
    return None

In [21]:
def decisions(t):
    global c, e
    
    tp = year(t, 0, 1)
    e[t] = (ξ*υ*wn[tp]/(wn[t]*(1.0-τs[t])*Rn[tp]))**(1.0/(1.0-υ))
    n_e[tp] = μ*(1.0+ξ*e[t]**υ) - 1.0
    
    c[0,t] = get_Psi(0,t)*get_W(0,t)
    get_path(0,t)
    
    if t == 1:
        for j in range(1,J):
            c[j,t] = get_Psi(j,t)*get_W(j,t)
            get_path(j,t)
    return None

In [22]:
## calculating quantities in a certain year

def quantities(t):
    global G, C, A, L, H, Y, B, K, Bf, Tb, I, ma
    
    tm = year(t,1,0)
    tp = year(t,0,1)
    
    if t == 0:
        G[t] = gy*Y[t]
    else:
        G[t] = G[0]
        
    ma[0,t] = 1.0
    for j in range(1,J):
        ma[j,t] = ma[j-1,tm]/((1.0+n_e[t])*(1.0+n_p[t]))
    
        
    # aggregate individual decisions
    C[t] = .0
    A[t] = .0
    L[t] = .0
    H[t] = .0
    
    for j in range(J):
        C[t] = C[t] + c[j,t]*ma[j,t]
        A[t] = A[t] + a[j,t]*ma[j,t]
        if j < JR-1:
            L[t] = L[t] + μ**(-j)*m[j,t]
            H[t] = H[t] + m[j,t]
            
    L[t] = L[t] - e[t]
    H[t] = L[t]/(H[t] - e[t])
    
    Y[t] = K[t]**α*L[t]**(1.0-α)*H[t]**ϵ
    B[t] = by[tm]*Y[t]
    
    # derive capital in small open or closed economy
    if smopec and t>0:
        K[t]  = L[t]*((r[t]*(1.0-eps[t]*τk[t])/(1.0-τk[t]) + δ)/α)**(1.0/(α-1.0)) 
        Bf[t] = A[t] - K[t] - B[t]
        Tb[t] = (1.0+n_p[tp])*(1.0+n_e[tp])*Bf[tp] - (1.0+r[t])*Bf[t]
    else:
        K[t] = damp*(A[t]-B[t]) + (1.0-damp)*K[t]
    
    I[t] = (1.0+n_p[tp])*(1.0+n_e[tp])*K[tp] - (1.0 - δ)*K[t]
    
    return None

In [23]:
# calculating government parameters

def government(t):
    global τc, τk, τw, τr, pen, Tpen, τp, TXr
    tp = year(t, 0, 1)
    
    taxrev = np.zeros(5)
    
    taxrev[0] = τc[t]*C[t]
    taxrev[1] = τw[t]*w[t]*L[t]
    taxrev[2] = τr[t]*r[t]*A[t]
    taxrev[3] = τk[t]*(Y[t] - w[t]*L[t]-(δ+eps[t]*r[t])*K[t])
    taxrev[4] = τs[t]*e[t]*wn[t]
    
    n_g = (1.0+n_p[tp])*(1.0+n_e[tp]) -1.0
    
    # get budget balancing tax rate
    if tax[t] == 1:      #(government surplus-τcC)/C
        τc[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[1] + taxrev[2] + \
                    taxrev[3]+(1.0 + n_g)*B[tp]))/C[t]   
    elif tax[t] == 2:    #[government surplus-τw(wL)-τr(rA)]/(wL+rA) assuming same labour and capital income tax
        τw[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[3] + \
                    (1.0+n_g)*B[tp]))/(w[t]*L[t] + r[t]*A[t])
        τr[t] = τw[t]    #same labour and capital income tax
    elif tax[t] == 3:    #[government surplus-τw(WL)]/(wL)
        τw[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[2] + \
                    taxrev[3] + (1.0 + n_g)*B[tp]))/(w[t]*L[t])
    else:                #[government surplus-τr(rA)]/(rA)
        τr[t] = (taxrev[4] + (1.0 + r[t])*B[t] + G[t] - (taxrev[0] + taxrev[1] + \
                    taxrev[3]+ (1.0 + n_g)*B[tp]))/(r[t]*A[t])
    
    TXr[t] = np.sum(taxrev[0:4]) - taxrev[4]
    
    # get budget balancing social security contribution
    pen[JR-1:J,t] = κ[t]*w[t]
    Tpen[t] = .0
    
    for j in range(JR-1,J):
        Tpen[t] = Tpen[t] + pen[j,t]*ma[j,t]
        
    τp[t] = Tpen[t]/(w[t]*L[t])
    return None

In [24]:
def get_SteadyState():
    K[0] = 1.0
    L[0] = 1.0
    for i in range(itermax+1):
        factor_prices(0)
        decisions(0)
        quantities(0)
        government(0)
        if abs(Y[0]-C[0]-I[0]-G[0])/Y[0] < tol:
            break
    if i < itermax:
        print('Iteration: ', i, ' Diff: ',abs(Y[0]-C[0]-I[0]-G[0])/Y[0])
    else:
        print('!!! No equilibrium found !!!')
    
    return None

In [25]:
## sovles for transition path using Gauss-Seidel:

def get_Transition():
    global a, K, L, HEV, H, h, Δ
    # Initialize values from initial equilibrium
    a[:,1] = a[:,0]
    n_e[1] = n_e[0]
    K[:] = K[0]
    L[:] = L[0]
    H[:] = H[0]
    
    for ite in range(itermax+2):
        # get prices, decisions and quantites
        for t in range(T+1):
            factor_prices(t)
        
        for t in range(T+1):
            decisions(t)
        
        for t in range(T+1):
            quantities(t)
        
        for t in range(T+1):
            government(t)
            
        # check for the number of markets in equilibrium
        nmarket = 0
        for t in range(1,T+1):
            if  abs(Y[t]-C[t]-I[t]-G[t]-Tb[t])/Y[t] < tol:
                nmarket = nmarket + 1
        if nmarket == T:
            break
        
    if ite > itermax:
        print('!!! No equilibrium found !!!')
    else:
        print('Iteration: ',ite,' Markets: ',nmarket,' Diff: ',np.max(np.abs(Y-C-I-G-Tb)/Y))
    return None

In [26]:
# Table 7.8
initialize()
gy = .0
μm = [1.2, 1.0, 0.8]
Cm = np.zeros(3);em = np.zeros(3);nem = np.zeros(3);Lm = np.zeros(3);Km = np.zeros(3)
wm = np.zeros(3);rm = np.zeros(3)

for i in range(3):
    μ = μm[i]
    get_SteadyState()
    Cm[i] = C[0];em[i] = e[0];nem[i] = n_e[0];Lm[i] = L[0];Km[i] = K[0];wm[i] = w[0]
    rm[i] = r[0]

print('--------------------------------------')
print('Table 7.8 Long-run analysis of\nhuman-capital acquisition')
print('--------------------------------------')
print(' μ    C    e   ne    L    K    w    r')
print('--------------------------------------')
for i in range(3):
    print('%.1f'%μm[i],'%.2f'%Cm[i],'%.2f'%em[i],'%.2f'%nem[i],'%.2f'%Lm[i],'%.2f'%Km[i],'%.2f'%wm[i],'%.2f'%rm[i])

Iteration:  32  Diff:  8.22941653956756e-07
Iteration:  30  Diff:  8.458596542501232e-07
Iteration:  28  Diff:  9.042060961971886e-07
--------------------------------------
Table 7.8 Long-run analysis of
human-capital acquisition
--------------------------------------
 μ    C    e   ne    L    K    w    r
--------------------------------------
1.2 0.62 0.07 0.86 1.76 0.08 0.27 2.66
1.0 0.73 0.09 0.60 1.91 0.10 0.29 2.31
0.8 0.91 0.11 0.34 2.14 0.15 0.31 1.96


In [27]:
# Table 7.9
initialize()
μ = .8
gy = .22
tax[1:] = 3
get_SteadyState()
get_Transition()
print('-----------------------------------------------')
print('Table 7.9 From consumption to labour-income\ntaxation with endogenous growth')
print('-----------------------------------------------')
print(' t   τw   τc    C    e   ne    L    K    w    r ')
print('-----------------------------------------------')
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%τw[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%n_e[t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t])
print(' .   .    .    .    .    .    .    .    .    .')
print(' .   .    .    .    .    .    .    .    .    .')
print(' .   .    .    .    .    .    .    .    .    .')
for t in [T]:
    print(' T','%.2f'%τw[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%n_e[t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t])
print('-----------------------------------------------')

Iteration:  29  Diff:  8.107432976544846e-07
Iteration:  34  Markets:  24  Diff:  8.656958906270022e-07
-----------------------------------------------
Table 7.9 From consumption to labour-income
taxation with endogenous growth
-----------------------------------------------
 t   τw   τc    C    e   ne    L    K    w    r 
-----------------------------------------------
 0 0.00 0.30 0.70 0.11 0.34 2.14 0.15 0.31 1.96
 1 0.31 0.00 0.75 0.08 0.34 2.17 0.15 0.31 1.98
 2 0.33 0.00 0.68 0.08 0.25 2.17 0.12 0.29 2.25
 3 0.34 0.00 0.66 0.08 0.26 2.17 0.11 0.29 2.36
 4 0.34 0.00 0.65 0.08 0.25 2.17 0.11 0.28 2.44
 .   .    .    .    .    .    .    .    .    .
 .   .    .    .    .    .    .    .    .    .
 .   .    .    .    .    .    .    .    .    .
 T 0.35 0.00 0.63 0.08 0.25 2.17 0.10 0.28 2.54
-----------------------------------------------


In [28]:
# Table 7.10
# Table 7.9
initialize()
μ = .8
gy = .22
τs[1:] = .25
get_SteadyState()
get_Transition()
print('-----------------------------------------------')
print('Table 7.10 Education subsidies with endogenous growth')
print('-----------------------------------------------')
print(' t   τs   τc    C    e   ne    L    K    w    r ')
print('-----------------------------------------------')
list1 = [0, 1, 2, 3, 4]
for t in list1:
    print('%2.f'%t,'%.2f'%τs[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%n_e[t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t])
print(' .   .    .    .    .    .    .    .    .    .')
print(' .   .    .    .    .    .    .    .    .    .')
print(' .   .    .    .    .    .    .    .    .    .')
for t in [T]:
    print(' T','%.2f'%τs[t],'%.2f'%τc[t],'%.2f'%C[t],'%.2f'%e[t],'%.2f'%n_e[t],'%.2f'%L[t],'%.2f'%K[t],'%.2f'%w[t],'%.2f'%r[t])
print('-----------------------------------------------')

Iteration:  29  Diff:  8.107432976544846e-07
Iteration:  19  Markets:  24  Diff:  8.203982977455009e-07
-----------------------------------------------
Table 7.10 Education subsidies with endogenous growth
-----------------------------------------------
 t   τs   τc    C    e   ne    L    K    w    r 
-----------------------------------------------
 0 0.00 0.30 0.70 0.11 0.34 2.14 0.15 0.31 1.96
 1 0.25 0.33 0.68 0.17 0.34 2.08 0.15 0.32 1.92
 2 0.25 0.34 0.65 0.17 0.46 2.08 0.13 0.30 2.09
 3 0.25 0.36 0.63 0.17 0.46 2.08 0.12 0.30 2.19
 4 0.25 0.36 0.62 0.17 0.46 2.08 0.12 0.30 2.23
 .   .    .    .    .    .    .    .    .    .
 .   .    .    .    .    .    .    .    .    .
 .   .    .    .    .    .    .    .    .    .
 T 0.25 0.36 0.61 0.17 0.45 2.08 0.12 0.29 2.27
-----------------------------------------------
